# Command-line Contra

### Aim:
Draw board in command-line type visualisation which can show beginning and end of contra dance move defined by user.

### Notes:
* I will be using camelCase for functions and variable.
* Position variables are zero indexed and the y-axis is top to bottom.



In [21]:
# imports and setup

import numpy as np
from pprint import pprint as pp

In [40]:
# define classes

class DanceFloor(object):
    """A schematic of a dance floor.
    
    Attributes:
        length: An integer giving the length of the dance floor
        width: An integer giving the width of the dance floor.
        floor: An array representing the dance floor.
        dancers: A dictionary with key-value pairs of dancers on the floor and their positions
        couples: A list of all couples on the dance floor.
        neighbours: A list of all neighbours on the dance floor
        sets: A list of all sets on the dance floor.
        
    """
    
    def __init__(self, length, width=8):
        self.length = length
        self.width = width
        try:
            self.floor = np.full((length, width), "_", dtype=str)
        except TypeError:
            print("Dimension argument(s) for DanceFloor object must be integers.")
        self.dancers = dict()
        self.couples = Pairs(self.floor)
        self.neighbours = Pairs(self.floor)
        self.ones = Pairs(self.floor)
        self.twos = Pairs(self.floor)
        
    
    
    def updateFloor(self):
        """Updates dancefloor after move"""
        # reset floor
        self.floor = np.full((self.length, self.width), "_", dtype=str)
        # update dancers' symbols on floor
        for dancer in self.dancers:
            self.floor[self.dancers[dancer].position[1], self.dancers[dancer].position[0]] = dancer
        
        
    def showFloor(self):
        """Shows the current state of the dance floor"""
        # draw floor
        for i in self.floor:
            print(i)
        print("")
        
        
    def listDancers(self):
        """Lists the dancers currently on the floor with their positions"""
        for index in self.dancers:
            print(index + " ~ " + str(self.dancers[index].position))
            
            
    def addDancer(self, dancer):
        """Adds a dancer to the floor at their starting position"""
        # check symbol not being used by another dancer
        if dancer.symbol not in self.dancers:
            # check symbol is single character
            if len(dancer.symbol) == 1:
                try:               
                    # check for another dancer in the same position
                    if self.floor[dancer.position[1], dancer.position[0]] == "_":
                        # change floor marker to dancer's symbol and update dict of dancers
                        self.dancers[dancer.symbol] = dancer
                        self.floor[dancer.position[1], dancer.position[0]] = dancer.symbol
                    else:
                        print("There is already a dancer at the position %s." % dancer.position)
                except IndexError:
                    print("The position %s is out of bounds." % dancer.position)
            else:
                print("Symbol for a dancer must be a single character.")
        else:
            print("The symbol '%s' is already being used for a dancer." % dancer.symbol)
        
      # not sure if I want this method  
#     def removeDancer(self, symbol):
#         """Removes a dancer from the floor, searching by symbol"""
#         try:
#             pos = self.dancers[symbol]
#             self.floor[pos[1], pos[0]] = "_"
#             del self.dancers[symbol]
#         except KeyError:
#             print("No dancer with that symbol. Currently, the dancers are: ", end="")
#             for key in self.dancers.keys():
#                 print(key, end=" ")
                
                
class Dancer(object):
    """ A sprite representing a dancer.
    
    Attributes:
        symbol: A character representing the dancer on the floor e.g. 'a', '1'.
        floor: The dance floor on which this dancer are dancing.
        position: A two-integer array giving the dancer's position e.g. [2,4].
        partner: The dancer who is this dancer's partner.
        neighbour: The dancer who is this dancer's neighbour.
    
    """
    def __init__(self, symbol, position, floor): 
        self.symbol = symbol
        self.floor = floor
        self.position = position
        self.partner = None
        self.neighbour = None
        self.floor.addDancer(self)  # add dancer to dance floor    
        
        
class Couple(object):
    """
    Two dancers who interact as a couple in a dance.
    
    Attributes:
        floor: The dance floor on which the couple are dancing
        d1: first dancer in the couple ('lead' place)
        d2: second dancer in couple ('follow' place)
    
    """
    def __init__(self, floor, d1sym, d1pos, d2sym, d2pos):
        self.floor = floor
        self.d1 = Dancer(d1sym, d1pos, floor)
        self.d2 = Dancer(d2sym, d2pos, floor)        
        self.d1.partner = self.d2
        self.d2.partner = self.d1
        self.floor.couples.append(self)
        
        
class Neighbour(object):
    """Two dancers who interact as neighbours in a dance.
    
    
    Attributes:
        floor: The dance floor on which the neighbours are dancing.
        d1: One of the neighbours.
        d2: The other neighbour.
        
    """
    def __init__(self, floor, d1, d2):
        self.floor = floor
        self.d1 = d1
        self.d2 = d2
        self.d1.neighbour = self.d2
        self.d2.neighbour = self.d1
        self.floor.neighbours.append(self)
        
        
class Pairs(object):
    """Pairs of dancers (e.g. neighbours, partners) who interact in the dance.
    
    Attributes:
        floor: The dance floor on which the pairs are dancing.
        pairs: A list of the pairs dancing
        
    """
    
    def __init__(self, floor):
        self.floor = floor
        self.pairs = []
        
        
    def append(self, pair):
        self.pairs.append(pair)
        
        
    def swapPlaces(self):
        """Move ending with the pair in each other's places"""
        for pair in self.pairs:
            pair.d1.position, pair.d2.position = pair.d2.position, pair.d1.position
        floor.updateFloor()
            
            
class Set(object):
    """A number of dancers (currently set at four) who interact as a set.
    
    Attributes:
        floor: The floor on which the set are dancing.
        first_lead_pos: A 2x2 array giving the starting position of the first couple's lead dancer (to position the set).
        one_l: Symbol representing the lead of the first couple.
        one_f: Symbol representing the follow of the first couple.
        two_l: Symbol representing the lead of the second couple.
        two_f: Symbol representing the follow of the second couple.
    
    """
        
    def __init__(self, floor, l1Pos, l1='A', f1='a', l2='B', f2='b', formation="Proper"):
        self.floor = floor
            
        # define relative positions
        if formation=="Proper":
            self.l1Pos = l1Pos
            self.f1Pos = list(np.array(l1Pos) + [3,0])
            self.l2Pos = list(np.array(l1Pos) + [0,1])
            self.f2Pos = list(np.array(l1Pos) + [3,1])
            
        else:
            # better error handling needed here
            print("No such formation.")
            exit(1)
            
        # instantiate couples
        self.ones = Couple(self.floor, l1, self.l1Pos, f1, self.f1Pos)
        self.floor.ones.append(self.ones)
        self.twos = Couple(self.floor, l2, self.l2Pos, f2, self.f2Pos)
        self.floor.twos.append(self.twos)
        
        # define neighbours; may be done with function later
        if formation=="Proper":
            self.neighbour1 = Neighbour(self.floor, self.ones.d1, self.twos.d1)
            self.neighbour2 = Neighbour(self.floor, self.ones.d2, self.twos.d2)
        else:
            print("No such formation.")
            exit(2)
            


In [41]:
floor = DanceFloor(8)
floor.showFloor()
mySet = Set(floor, [2,3])
floor.showFloor()
floor.listDancers()

['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']

['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' 'A' '_' '_' 'a' '_' '_']
['_' '_' 'B' '_' '_' 'b' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']

A ~ [2, 3]
a ~ [5, 3]
B ~ [2, 4]
b ~ [5, 4]
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' 'a' '_' '_' 'A' '_' '_']
['_' '_' 'B' '_' '_' 'b' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']



In [6]:
# # testing dance floor class
# floor = DanceFloor(8)     
# couple1 = Couple(floor, 'A', [2,3], 'a', [5,3])
# couple2 = Couple(floor, 'B', [2,4], 'b', [5,4])
# couples = floor.couples

# floor.showFloor()
# floor.listDancers()
# for couple in couples.pairs:
#     print(couple.d1.position, couple.d2.position)
    
# # getting Pairs class to work

# couples.swapPlaces()

# floor.showFloor()

['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' 'A' '_' '_' 'a' '_' '_']
['_' '_' 'B' '_' '_' 'b' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']

A ~ [2, 3]
a ~ [5, 3]
B ~ [2, 4]
b ~ [5, 4]
[2, 3] [5, 3]
[2, 4] [5, 4]
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' 'a' '_' '_' 'A' '_' '_']
['_' '_' 'b' '_' '_' 'B' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']



In [7]:
# # testing switching moves
# # must call 'floor.updateFloor()' after each move

# def swapPlaces(d1, d2):
#     d1.position, d2.position = d2.position, d1.position
#     floor.updateFloor()
#     return d1, d2


# swapPlaces(couple1.d1, couple1.d2)

# floor.showFloor()

# # :-D

['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' 'A' '_' '_' 'a' '_' '_']
['_' '_' 'b' '_' '_' 'B' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']



In [11]:
# try to call swapPlaces on all couples on floor

floor.showFloor()
couples = floor.couples

couples.swapPlaces
floor.showFloor()

['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' 'A' '_' '_' 'a' '_' '_']
['_' '_' 'B' '_' '_' 'b' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']

['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' 'A' '_' '_' 'a' '_' '_']
['_' '_' 'B' '_' '_' 'b' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']

